In [3]:
import psycopg

# Connect to your PostgreSQL database
# Make sure it is already running on your computer.
# (This is done by default on JupyterHub)
conn = psycopg.connect("dbname=yelp_dataset host=localhost")



# Create a cursor object
cur = conn.cursor()

#  Create Tables
cur.execute("""
    CREATE TABLE IF NOT EXISTS businesses (
        business_id VARCHAR PRIMARY KEY,
        name VARCHAR,
        city VARCHAR,
        stars DECIMAL,
        review_count INT,
        categories TEXT
    )
""")


cur.execute("""
    CREATE TABLE IF NOT EXISTS reviews (
        review_id VARCHAR PRIMARY KEY,
        business_id VARCHAR REFERENCES businesses(business_id),
        user_id VARCHAR,
        stars DECIMAL,
        text TEXT,
        date DATE
    )
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS users (
        user_id VARCHAR PRIMARY KEY,
        name VARCHAR,
        review_count INT,
        avg_stars DECIMAL
    )
""")

# Commit the transaction
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

In [4]:
cur.execute("TRUNCATE TABLE businesses CASCADE;")
cur.execute("TRUNCATE TABLE reviews CASCADE;")
cur.execute("TRUNCATE TABLE users CASCADE;")

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block

In [15]:
import psycopg
import json
import random


business_file = "/home/jovyan/data101finalproj/data/yelp_academic_dataset_business.json"
review_file = "/home/jovyan/data101finalproj/data/yelp_academic_dataset_review.json"
user_file = "/home/jovyan/data101finalproj/data/yelp_academic_dataset_user.json"

# connect to your PostgreSQL database
conn = psycopg.connect("dbname=yelp_dataset host=localhost")
cur = conn.cursor()

#so we can start fresh with every time we run this code
cur.execute("TRUNCATE TABLE businesses CASCADE;")
cur.execute("TRUNCATE TABLE reviews CASCADE;")
cur.execute("TRUNCATE TABLE users CASCADE;")

# load and sample businesses
with open(business_file, 'r') as f:
    businesses = [json.loads(line) for line in f]

sampled_businesses = random.sample(businesses, 10000)

business_data = [
    (
        business["business_id"],
        business.get("name", None),
        business.get("city", None),
        business.get("stars", None),
        business.get("review_count", None),
        business.get("categories", None)
    )
    for business in sampled_businesses
]

# put sampled businesses into  database

cur.executemany("""
    INSERT INTO businesses (business_id, name, city, stars, review_count, categories)
    VALUES (%s, %s, %s, %s, %s, %s)
""", business_data)

#load reviews and filter for sampled businesses

with open(review_file, 'r') as f:
    reviews = [json.loads(line) for line in f]

sampled_business_ids = set(b["business_id"] for b in sampled_businesses)
filtered_reviews = [review for review in reviews if review["business_id"] in sampled_business_ids]


# filter users for the filtered reviews

sampled_user_ids = set(review["user_id"] for review in filtered_reviews)

with open(user_file, 'r') as f:
    users = [json.loads(line) for line in f]

filtered_users = [user for user in users if user["user_id"] in sampled_user_ids]

#insert filtered users

user_data = [
    (
        user["user_id"],
        user.get("name", None),
        user.get("review_count", None),
        user.get("average_stars", None)
    )
    for user in filtered_users
]

cur.executemany("""
    INSERT INTO users (user_id, name, review_count, avg_stars)
    VALUES (%s, %s, %s, %s)
""", user_data)

# filter reviews again for valid users 

valid_user_ids = set(user["user_id"] for user in filtered_users)
filtered_reviews = [
    review for review in filtered_reviews if review["user_id"] in valid_user_ids
]

# insert filtered reviews
review_data = [
    (
        review["review_id"],
            review["business_id"],
        review["user_id"],
        review.get("stars", None),
        review.get("text", None),
        review.get("date", None)
    )
    for review in filtered_reviews
]

cur.executemany("""
    INSERT INTO reviews (review_id, business_id, user_id, stars, text, date)
    VALUES (%s, %s, %s, %s, %s, %s)
""", review_data)


conn.commit()
cur.close()
conn.close()
print("Data loading complete")


Data loading complete


In [18]:
import psycopg


conn = psycopg.connect("dbname=yelp_dataset host=localhost")
cur = conn.cursor()


cur.execute("SELECT * FROM businesses LIMIT 10;")
businesses = cur.fetchall()
print("Businesses:")
for row in businesses:
    print(row)

cur.execute("SELECT * FROM reviews LIMIT 10;")
reviews = cur.fetchall()
print("Reviews:")
for row in reviews:
    print(row)


cur.execute("SELECT * FROM users LIMIT 10;")
users = cur.fetchall()
print("Users:")
for row in users:
    print(row)


cur.close()
conn.close()


Businesses:
('P5zFxurVra136fM4-uJhjA', 'Lone Star Steakhouse', 'Orlando', Decimal('3.5'), 123, 'American (Traditional), Seafood, Steakhouses, Restaurants')
('rlj2exPbE-TtqVb7rtdAJA', 'Burger King', 'Aloha', Decimal('1.5'), 17, 'Fast Food, Restaurants, Burgers')
('I38xc3pf69Hfv9V8xFT6PA', 'Old Navy', 'Bee Cave', Decimal('2.5'), 9, "Fashion, Men's Clothing, Children's Clothing, Shopping, Women's Clothing")
('0A9TFJ4Y_D63OpELgsST6g', 'Christine Norcross & Partners', 'Wellesley', Decimal('4'), 5, 'Home Services, Real Estate Agents, Real Estate')
('ximuqcT7oYTT3FT2vRtq5w', 'Audi North Austin', 'Austin', Decimal('2.5'), 189, 'Auto Parts & Supplies, Automotive, Car Dealers, Auto Repair, Used Car Dealers')
('0Q9zwx6DUlB6htkliT5eNw', "The Children's Courtyard", 'Austin', Decimal('2.5'), 8, 'Elementary Schools, Education, Local Services, Preschools, Tutoring Centers, Child Care & Day Care')
('rnbAaSZzj1exmXOhiNuogg', "Miss Emily's Bed & Biscuit", 'Orlando', Decimal('4.5'), 45, 'Pet Sitting, Pet 